<a href="https://colab.research.google.com/github/Pegasus-nathanael/Test_Nathan/blob/main/Scrapping_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [70]:
!pip install beautifulsoup4
!pip install requests
!pip install lxml
!pip install html5lib

In [71]:
from bs4 import BeautifulSoup
import requests
import lxml
import pandas as pd
import time
import random as ran
import sys

In [72]:
url='https://www.imdb.com/search/title/?release_date=2022-01-01,2022-12-31&sort=boxoffice_gross_us,desc'

In [73]:
headers = {
       'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36'
   }

In [74]:
source = requests.get(url, headers=headers)

In [75]:
source

<Response [200]>

In [ ]:
soup = BeautifulSoup(source.content,'lxml')

In [ ]:
# Récupération des blocks de films

blocs_de_films = soup.find_all('div', {'class': 'sc-f30335b4-0 eefKuM'}) # La variable `blocs_de_films` stocke une liste de tous les blocs de films sur la page. Un bloc de film est un élément HTML qui contient des informations sur un film, telles que le titre, l'année de sortie, le classement IMDb et l'affiche.

In [ ]:
# Récupérer l'URL du film
url_film = blocs_de_films[0].find('a')['href']

In [ ]:
if blocs_de_films:
    url_film = blocs_de_films[0].find('a')['href']
    print(url_film)  # Affichez l'URL du film
else:
    print("Aucun bloc de film trouvé.")

In [ ]:
if blocs_de_films:
    url_film = 'https://www.imdb.com' + blocs_de_films[0].find('a')['href']
    print(url_film)
else:
    print("Aucun bloc de film trouvé.")

In [ ]:
# Récupération des blocks de films
blocs_de_films = soup.find_all('div', {'class': 'sc-f30335b4-0 eefKuM'})

# Extraction des informations pour le premier film (blocs_de_films[0])
nom_film = blocs_de_films[0].find('a').text
annee_film = blocs_de_films[0].find_all('span', class_='sc-f30335b4-7')[0].text
duree_film = blocs_de_films[0].find_all('span', class_='sc-f30335b4-7')[1].text
genre = blocs_de_films[0].find_all('span', class_='sc-f30335b4-7')[2].text
note_imdb = blocs_de_films[0].find('span', class_='ipc-rating-star--rating').text
meta_score = blocs_de_films[0].find('span', class_='sc-b0901df4-0 bXIOoL metacritic-score-box').text

print(f"Nom du film: {nom_film}")
print(f"Année de sortie: {annee_film}")
print(f"Durée du film: {duree_film}")
print(f"Genre: {genre}")
print(f"Note IMDB: {note_imdb}")
print(f"MetaScore: {meta_score}")

In [ ]:
# Afficher tous les détails du film
print("URL du film : " + url_film,
      "\nNom du film : " + nom_film,
      "\nAnnée de sortie du film : " + str(annee_film),
      "\nGenre : " + genre,
      "\nNote IMDB : " + str(note_imdb),
      "\nMetaScore : " + str(meta_score)
      )

In [ ]:
def recuperer_details_film(blocs_de_films):
    """
    Récupère les détails d'un film à partir du bloc de données HTML fourni.

    Paramètres :
        blocs_de_films (bs4.element.Tag) : Le bloc HTML contenant les détails du film.

    Retourne :
        dict : Un dictionnaire contenant les détails du film. Les clés du dictionnaire sont :
            - 'nom' : Le nom unique du film.
            - 'annee_sortie' : L'année de sortie du film.
            - 'genre' : Le genre du film.
            - 'duree' : La durée du film.
            - 'note_imdb' : La note IMDB du film.
    """

    # Initialiser un dictionnaire pour stocker les données du film
    donnees_film = {}

    # Récupérer le nom du film
    try:
        donnees_film['nom'] = blocs_de_films.find('a').get_text()
    except:
        donnees_film['nom'] = None

    # Récupérer l'année de sortie du film
    try:
        donnees_film['annee_sortie'] = blocs_de_films.find_all('span', class_='sc-f30335b4-7')[0].text
    except:
        donnees_film['annee_sortie'] = None

    # Récupérer le genre du film
    try:
        donnees_film['genre'] = blocs_de_films.find_all('span', class_='sc-f30335b4-7')[2].text
    except:
        donnees_film['genre'] = None

    # Récupérer la durée du film
    try:
        donnees_film['duree'] = blocs_de_films.find_all('span', class_='sc-f30335b4-7')[1].text
    except:
        donnees_film['duree'] = None

    # Récupérer la note IMDb du film
    try:
        donnees_film['note_imdb'] = blocs_de_films.find('span', class_='ipc-rating-star--rating').text
    except:
        donnees_film['note_imdb'] = None
    try:
        donnees_film['meta_score'] = blocs_de_films.find('span', class_='sc-b0901df4-0 bXIOoL metacritic-score-box').text
    except:
        donnees_film['meta_score'] = None

    # Retourner le dictionnaire contenant les détails du film
    return donnees_film

In [ ]:
def extraire_donnees_page_films(blocs_de_films):
    """
    Extrait les détails de chaque film à partir des blocs de données HTML de la page.

    Paramètres :
        blocs_de_films (list) : Une liste des blocs HTML contenant les détails de chaque film.

    Retourne :
        list : Une liste de dictionnaires, chaque dictionnaire contenant les détails d'un film.
            Les clés du dictionnaire sont :
            - 'nom' : Le nom unique du film.
            - 'annee_sortie' : L'année de sortie du film.
            - 'genre' : Le genre du film.
            - 'duree' : La durée du film.
            - 'note_imdb' : La note IMDB du film.
    """

    liste_donnees_films = []  # Initialiser une liste pour stocker les détails de chaque film
    nb_blocs = len(blocs_de_films)

    for i in range(nb_blocs):
        # Appeler la fonction récupérant les détails du film pour chaque bloc de films
        donnees_film = recuperer_details_film(blocs_de_films[i])

        # Ajouter les détails du film à la liste des données de la page
        liste_donnees_films.append(donnees_film)

    return liste_donnees_films

In [ ]:
def extraire_donnees_films_par_lien(url, cible):
    """
    Extrait les détails des films à partir d'un lien donné jusqu'à atteindre la cible de films.

    Paramètres :
        url (str) : L'URL de la page à partir de laquelle extraire les détails des films.
        cible (int) : Le nombre total de films que l'on souhaite extraire.

    Retourne :
        list : Une liste de dictionnaires, chaque dictionnaire contenant les détails d'un film.
            Les clés du dictionnaire sont :
            - 'nom' : Le nom unique du film.
            - 'annee_sortie' : L'année de sortie du film.
            - 'genre' : Le genre du film.
            - 'duree' : La durée du film.
            - 'note_imdb' : La note IMDB du film.
    """

    url_base = url
    cible_films = cible

    m_compte_debut_actuel = 0
    m_compte_fin_actuel = 0
    m_compte_restant = cible_films - m_compte_fin_actuel

    numero_nouvelle_page = 1

    donnees_films = []  # Initialiser une liste pour stocker les détails de chaque film

    while m_compte_restant > 0:
        url_complete = url_base + '&start=' + str(numero_nouvelle_page) # construct the full URL using '&start=' for pagination

        # Récupérer le contenu HTML de la page
        source = requests.get(url_complete).text
        soup = BeautifulSoup(source, 'html.parser')

        # Trouver tous les blocs de films sur la page
        blocs_de_films = soup.find_all('div', {'class': 'sc-f30335b4-0 eefKuM'}) # Use "lister-item mode-advanced" as the selector
        print(blocs_de_films)

        # Appeler la fonction extraire_donnees_page_films pour chaque page de films
        donnees_page_films = extraire_donnees_page_films(blocs_de_films)

        # Ajouter les détails des films à la liste de données des films
        donnees_films.extend(donnees_page_films)

        # Mettre à jour les compteurs actuels et le nombre de films restants à extraire
        # Extract the relevant pagination information:

        # Find the navigation bar <div class="desc"> element
        desc_div = soup.find("span", {"class": "ipc-see-more__text"})
        print(desc_div)
        if desc_div: # Make sure desc_div exists before proceeding
            # Get the text content of the desc_div
            desc_text = desc_div.get_text(strip=True)
            # Try to extract the start and end values
            try:
                m_compte_debut_actuel, m_compte_fin_actuel = map(int, desc_text.split("–")[0].split("-")[0].strip().split(" ")[-1], desc_text.split("–")[1].split(" ")[0].strip())
            except (IndexError, ValueError):
                # Error handling if the formatting has changed. Stop the scraping.
                print(f"Unable to parse pagination information from '{desc_text}'")
                break
            m_compte_restant = cible_films - m_compte_fin_actuel
        else:
            print("Pagination information not found.")
            break

        # Afficher l'état actuel de l'extraction des films
        print('\r' + "Extraction des films en cours : " + str(m_compte_debut_actuel) + " - "+ str(m_compte_fin_actuel), "| Restant : " + str(m_compte_restant), flush=True, end="")

        numero_nouvelle_page += 50 # Update pagination according to IMDB structure

        # Attendre un certain temps aléatoire avant de récupérer la prochaine page (pour éviter de surcharger le serveur)
        time.sleep(ran.randint(0, 10))

    return donnees_films

In [ ]:
base_scraping_link = "https://www.imdb.com/search/title/?title_type=feature&release_date=2022-01-01,2022-12-31&sort=boxoffice_gross_us,desc"

top_movies = 300  # Nombre de films à extraire

# Appeler la fonction pour extraire les détails des films à partir du lien de base et avec le nombre de films cible
films = extraire_donnees_films_par_lien(base_scraping_link, top_movies)

In [ ]:
movies_df.to_csv('movies_df.csv')